In [59]:
import tensorflow as tf
import time

In [60]:
tf.__version__

'2.6.0'

In [61]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )


In [62]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [63]:
%%timeit
benchmark(FileDataset())

282 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%%timeit
benchmark(FileDataset().prefetch(1))

245 ms ± 5.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

244 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [66]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [67]:
# Subsequent iterations read from the cache.
list(dataset.as_numpy_iterator())


[0, 1, 4, 9, 16]

In [68]:
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [69]:

%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

1.28 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [70]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

456 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
